# Level 12: Best Practices & Real-World Usage

Moving from learning examples to building real applications requires adopting best practices that ensure your code is robust, maintainable, and efficient. This notebook covers essential patterns and tools for using SQLAlchemy in production environments.

### Setup
We'll use a simple User model for the examples.

In [1]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker
import os

db_file = 'sqlalchemy_best_practices.db'
if os.path.exists(db_file):
    os.remove(db_file)

engine = create_engine(f'sqlite:///{db_file}')
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

## 12.1 Use Context Managers for Sessions

Manually calling `session.commit()`, `session.rollback()`, and `session.close()` can be error-prone. A much safer and cleaner pattern is to manage the session's lifecycle with a context manager. This ensures that the session is always closed and transactions are handled correctly, even if errors occur.

In [2]:
from contextlib import contextmanager

@contextmanager
def get_session():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    print("--- Session opened ---")
    try:
        yield session
        session.commit()
        print("--- Session committed ---")
    except Exception:
        session.rollback()
        print("--- Session rolled back ---")
        raise
    finally:
        session.close()
        print("--- Session closed ---")

# Usage Example 1: Success
with get_session() as session:
    session.add(User(name='Alice'))

# Usage Example 2: Failure
try:
    with get_session() as session:
        session.add(User(name='Bob'))
        raise ValueError("Something went wrong!")
except ValueError as e:
    print(f"Caught expected error: {e}")

--- Session opened ---
--- Session committed ---
--- Session closed ---
--- Session opened ---
--- Session rolled back ---
--- Session closed ---
Caught expected error: Something went wrong!


## 12.2 Connection Pooling

Establishing a new connection to a database for every request is very expensive and slow. **Connection Pooling** is a technique used to maintain a 'pool' of open database connections that can be reused.

**The good news:** SQLAlchemy's `Engine` manages a connection pool for you by default! When you call `engine.connect()` or use a `Session`, you are checking out a connection from this pool. When the connection is closed, it's returned to the pool, not actually terminated.

For most applications, the default settings are fine. For high-traffic applications, you might need to configure parameters like `pool_size` and `max_overflow` in `create_engine`.

## 12.3 Migrations with Alembic

What happens when you need to change your database schema after your application is already running with data? For example, you need to add a new `email` column to your `User` model.

Simply changing the model and running `Base.metadata.create_all(engine)` won't work, as it doesn't modify existing tables. This is where a **database migration tool** like **Alembic** comes in.

**Alembic** is the official migration tool for SQLAlchemy. It allows you to:
1.  **Track Schema Changes:** It compares your SQLAlchemy models to the current state of the database.
2.  **Generate Migration Scripts:** It automatically generates Python scripts that contain the `ALTER TABLE` commands needed to update the database.
3.  **Version Control Your Database:** You can upgrade your database to a newer version or downgrade it to an older one by applying these scripts.

Alembic is a command-line tool and its usage is beyond the scope of this notebook, but it is an essential part of any production SQLAlchemy application.

**Learn more:** [Alembic Official Website](https://alembic.sqlalchemy.org/)

## 12.4 Logging & Debugging

Sometimes, you need to see the exact SQL that SQLAlchemy is generating. You can enable this by setting `echo=True` on the engine.

In [3]:
# Create an engine with SQL echoing turned on
echo_engine = create_engine(f'sqlite:///{db_file}', echo=True)

EchoSession = sessionmaker(bind=echo_engine)
echo_session = EchoSession()

# Now, when we perform a query, the generated SQL will be printed to the console
user = echo_session.query(User).first()

echo_session.close()

2025-08-15 23:20:54,435 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-15 23:20:54,440 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name 
FROM users
 LIMIT ? OFFSET ?
2025-08-15 23:20:54,441 INFO sqlalchemy.engine.Engine [generated in 0.00160s] (1, 0)
2025-08-15 23:20:54,445 INFO sqlalchemy.engine.Engine ROLLBACK


### Viewing a Query's SQL
You can also see the SQL for a specific query object by converting it to a string.

In [4]:
query = session.query(User).filter(User.name.like('A%'))
print(str(query))

SELECT users.id AS users_id, users.name AS users_name 
FROM users 
WHERE users.name LIKE ?
